# Selection Predictive Value Testing

This notebook measures how well our daily selection lists foresee future breakouts. We’ll join historical selections with realized forward returns and compute precision/recall metrics for big moves (e.g., 10×), along with portfolio-style performance diagnostics.


## Workflow Overview
1. **Load selection log** – historical picks and scores per rebalance date.
2. **Compute future returns** – join QC history to calculate 7d/30d forward returns.
3. **Label outcomes** – classify 10×, breakouts, or neutral.
4. **Evaluate precision/recall** – check how many hits we captured vs. misses.
5. **Portfolio backtest** – compare selection returns against baselines.
6. **Diagnostics** – highlight misses, false positives, and component contributions.


In [ ]:
from datetime import timedelta
from pathlib import Path

import pandas as pd

try:
    from QuantConnect import Resolution  # type: ignore[import]
    from QuantConnect.Research import QuantBook  # type: ignore[import]
except ImportError as exc:
    raise RuntimeError("Run inside a QuantConnect Research environment to evaluate selection accuracy.") from exc

qb = QuantBook()
SELECTION_LOG_FILE = Path("data/native_features/selection_log.parquet")
FORWARD_WINDOWS = {"ret_7d": 7, "ret_30d": 30}


In [ ]:
UNIVERSE_LOG_FILE = Path("data/native_features/universe_log.parquet")

missing_files = [path for path in [SELECTION_LOG_FILE, UNIVERSE_LOG_FILE] if not path.exists()]
if missing_files:
    raise FileNotFoundError(
        "Missing data logs: " + ", ".join(str(path) for path in missing_files) + ". Run the selection notebook to generate them."
    )

selection_log = pd.read_parquet(SELECTION_LOG_FILE)
universe_log = pd.read_parquet(UNIVERSE_LOG_FILE)

for df in (selection_log, universe_log):
    df["date"] = pd.to_datetime(df["date"], utc=True)

selection_log = selection_log.sort_values("date")
universe_log = universe_log.sort_values("date")
selection_log.tail()


In [ ]:
def compute_forward_returns(symbol, dates, windows):
    quant_symbol = qb.AddCrypto(symbol, Resolution.Hour).Symbol
    history = qb.History(quant_symbol, dates.min(), dates.max() + timedelta(days=max(windows.values())), Resolution.Hour)
    if history.empty:
        return pd.DataFrame()
    closes = history.close.unstack(level=0).tz_localize(None)[quant_symbol.Value]
    returns = pd.DataFrame(index=dates)
    for label, days in windows.items():
        shifted = closes.shift(-days * 24)
        aligned = pd.concat([closes, shifted], axis=1).reindex(index=dates)
        returns[label] = (aligned.iloc[:, 1] / aligned.iloc[:, 0]) - 1
    returns.index.name = "date"
    returns["symbol"] = symbol
    return returns.reset_index()

forward_frames = []
for symbol in selection_log["symbol"].unique():
    dates = selection_log.loc[selection_log["symbol"] == symbol, "date"]
    frame = compute_forward_returns(symbol, dates, FORWARD_WINDOWS)
    if not frame.empty:
        forward_frames.append(frame)

forward_returns = (
    pd.concat(forward_frames, axis=0, ignore_index=True)
    if forward_frames
    else pd.DataFrame(columns=["date", "symbol", *FORWARD_WINDOWS.keys()])
)
selection_with_returns = selection_log.merge(forward_returns, on=["date", "symbol"], how="left")

universe_frames = []
for symbol in universe_log["symbol"].unique():
    dates = universe_log.loc[universe_log["symbol"] == symbol, "date"]
    frame = compute_forward_returns(symbol, dates, FORWARD_WINDOWS)
    if not frame.empty:
        universe_frames.append(frame)

universe_forward_returns = (
    pd.concat(universe_frames, axis=0, ignore_index=True)
    if universe_frames
    else pd.DataFrame(columns=["date", "symbol", *FORWARD_WINDOWS.keys()])
)
universe_with_returns = universe_log.merge(universe_forward_returns, on=["date", "symbol"], how="left")

selection_with_returns.tail()


In [ ]:
def label_breakouts(row, thresholds):
    if row["ret_30d"] >= thresholds.get("ten_x", 10.0):
        return "10x"
    if row["ret_30d"] >= thresholds.get("breakout", 2.0):
        return "breakout"
    if row["ret_7d"] >= thresholds.get("short_term", 0.5):
        return "short_surge"
    return "neutral"

THRESHOLDS = {"ten_x": 10.0, "breakout": 2.0, "short_term": 0.5}
for df in (selection_with_returns, universe_with_returns):
    df["label"] = df.apply(label_breakouts, axis=1, thresholds=THRESHOLDS)
    df["hit"] = df["label"].isin(["10x", "breakout"])

selection_with_returns.tail()


In [ ]:
precision = selection_with_returns["hit"].mean()

hits_total = universe_with_returns[universe_with_returns["hit"]]
hits_with_flags = pd.DataFrame()
if hits_total.empty:
    recall = 0.0
else:
    selected_pairs = selection_with_returns[["date", "symbol"]].copy()
    selected_pairs["selected"] = True
    hits_with_flags = hits_total.merge(selected_pairs, on=["date", "symbol"], how="left")
    hits_captured = hits_with_flags["selected"].fillna(False).sum()
    recall = hits_captured / len(hits_total)

precision, recall


In [ ]:
missed_hits = hits_with_flags[hits_with_flags["selected"].fillna(False) == False] if not hits_with_flags.empty else pd.DataFrame()
missed_hits.head()


### Notes & Next Steps
- Recall now uses `universe_log.parquet`; ensure the selection notebook keeps that log up-to-date (e.g., at each rebalance).
- Extend the diagnostics with:
  - Confusion matrices per threshold (10×, breakout, short surge).
  - Rolling precision/recall plots to monitor drift.
  - Portfolio backtests comparing selection vs. baseline baskets.
  - Hit/miss breakdowns by feature or component to guide score tuning.
- Consider caching forward returns to avoid repeated history pulls during development.
- Wire these diagnostics back into the main selection notebook so each research iteration automatically updates the predictive scorecard.
